In [1]:
import sys
import cv2
import mediapipe as mp
import glob
import numpy as np
import json
import os
import time
from os.path import exists

In [11]:
# Data paths
json_file = '/mnt/HDD04/WASL Dataset/WLASL-master/start_kit/WLASL_v0.3.json'
txt_file = 'files10.txt'
files_to_process = []

with open(json_file) as ipf:
    content = json.load(ipf)

file1 = open(txt_file, 'r')
temp = file1.readlines()
files_to_process = [t[:-1] for t in temp]


# datapath = '/mnt/HDD04/WASL Dataset/WLASL-master/start_kit/videos/'
datapath = '/mnt/HDD04/WASL Dataset/WLASL2000/'
vid_savepath = '/mnt/HDD04/WASL Dataset/Scripts/annotated_videos/'
annot_path = '/mnt/HDD04/WASL Dataset/Scripts/annotated_images/'
skelpath = '/mnt/HDD04/WASL Dataset/Scripts/skeletons/'
frame_path = skelpath.replace('skeletons', 'images')
json_path = skelpath.replace('skeletons', 'jsons')

In [12]:
files = glob.glob(datapath + '*mp4')
print('Num files present:', len(files))
print('Num files should be:', sum([len(c['instances']) for c in content]))
print(files[0])

Num files present: 21095
Num files should be: 21083
/mnt/HDD04/WASL Dataset/WLASL2000/23865.mp4


In [13]:
print('Class:', content[0]['gloss'])
print('sample:', content[0]['instances'][0])

Class: book
sample: {'bbox': [385, 37, 885, 720], 'fps': 25, 'frame_end': -1, 'frame_start': 1, 'instance_id': 0, 'signer_id': 118, 'source': 'aslbrick', 'split': 'train', 'url': 'http://aslbricks.org/New/ASL-Videos/book.mp4', 'variation_id': 0, 'video_id': '69241'}


In [14]:
content[0]['instances'][0]['video_id']

'69241'

In [15]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [17]:
# extract skeleton
start_time = time.time()
cnt = 0
for i, c in enumerate(content):
    for j in range(len(c['instances'])):
        if c['instances'][j]['video_id'] not in files_to_process:
            continue
        gloss = c['gloss']  # i.e., class 
        fname = datapath + c['instances'][j]['video_id'] + '.mp4'
        
        if exists(fname):
            cnt += 1
            now = time.time()
            print('Processing content ' + str(i+1) + '/' + str(len(content)) +
                  ', instance ' + str(j+1) + '/' + str(len(c['instances'])))
            cap = cv2.VideoCapture(fname)
            fps = cap.get(cv2.CAP_PROP_FPS)
#             start_frame = c['instances'][j]['frame_start'] - 1
            start_frame = 0
            end_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
#             if c['instances'][j]['frame_end'] != -1:
#                 end_frame = c['instances'][j]['frame_end']
            subfolder_frame = frame_path + fname.split('/')[-1][:-4]
            if not exists(subfolder_frame):
                os.mkdir(subfolder_frame)
                bbox = c['instances'][j]['bbox']
                # extract frames
    #             for frame_no in range(start_frame, end_frame + 1):
                for frame_no in range(start_frame, end_frame + 1):
                    cap.set(1,frame_no)  # Where frame_no is the frame you want
                    ret, frame = cap.read()  # Read the frame
#                     frame = frame[bbox[1]:bbox[3], bbox[0]:bbox[2]]
                    cv2.imwrite(subfolder_frame + '/' + fname.split('/')[-1][:-4] + '_frame' + f'{frame_no:05d}' + '.jpg', frame)
            
            # MediaPipe
            subfolder_skel = skelpath + fname.split('/')[-1][:-4]
            if not exists(subfolder_skel):
                os.mkdir(subfolder_skel)
                IMAGE_FILES = sorted(glob.glob(subfolder_frame + '/*jpg'))
                with mp_hands.Hands(
                    static_image_mode=True,
                    max_num_hands=2,
                    min_detection_confidence=0.5) as hands:

                    image = cv2.flip(cv2.imread(IMAGE_FILES[0]), 1)
                    fname2 = IMAGE_FILES[0].split('/')[-1]
                    prev_results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

                    for idx, file in enumerate(IMAGE_FILES):
                        # Read an image, flip it around y-axis for correct handedness output (see
                        # above).
                        image = cv2.flip(cv2.imread(file), 1)
                        fname2 = file.split('/')[-1]
                        # Convert the BGR image to RGB before processing.
                        results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

                        # Print handedness and draw hand landmarks on the image.
    #                     print('Frame: ', idx, '/', len(IMAGE_FILES))
                        # print('Handedness:', results.multi_handedness)
                        if not results.multi_hand_landmarks:
                            if not prev_results.multi_hand_landmarks:
                                continue
                            else:
                                results = prev_results  # to prevent frame elimination due to non-existent hands
                        prev_results = results
                        image_height, image_width, _ = image.shape
                        annotated_image = image.copy()
                        data = []

                        for hand_id, hand_landmarks in enumerate(results.multi_hand_landmarks):


                          # print('hand_landmarks:', hand_landmarks)
                          # print(
                          #     f'Index finger tip coordinates: (',
                          #     f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
                          #     f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height})'
                          # )
                          # print(dir(hand_landmarks.landmark[0]))
                          # print(hand_landmarks.landmark[0])
                          # print(hand_landmarks.landmark[0].x)

                            for land_id, mark in enumerate(hand_landmarks.landmark):

                                if land_id == 0:
                                    x_temp = [mark.x]
                                    y_temp = [mark.y]
                                    z_temp = [mark.z]
                                else:
                                    x_temp = np.concatenate((x_temp, [mark.x]), 0)
                                    y_temp = np.concatenate((y_temp, [mark.y]), 0)
                                    z_temp = np.concatenate((z_temp, [mark.z]), 0)

                            data.append(x_temp)
                            data.append(y_temp)
                            data.append(z_temp)
                            mp_drawing.draw_landmarks(
                                  annotated_image,
                                  hand_landmarks,
                                  mp_hands.HAND_CONNECTIONS,
                                  mp_drawing_styles.get_default_hand_landmarks_style(),
                                  mp_drawing_styles.get_default_hand_connections_style())
                        

                        with open(subfolder_skel + '/' + fname2.replace('jpg', 'txt'), 'w') as f:
                            for elem in data:
                                for e in elem:
                                    f.write(str(e) + ' ')
                                f.write('\n')
                        subfolder_im = annot_path + fname.split('/')[-1][:-4]
                        if not exists(subfolder_im):
                            os.mkdir(subfolder_im)
                        cv2.imwrite(subfolder_im + '/' + fname2.replace('jpg', 'png'), cv2.flip(annotated_image, 1))
                    
                    # create video
#                     files = sorted(glob.glob(subfolder_im + '/*png'))
#                     frame = cv2.imread(files[0])
#                     height, width, layers = frame.shape
#                     video_name = vid_savepath + fname.split('/')[-1][:-4] + '.avi'
#                     video = cv2.VideoWriter(video_name, 0, fps, (width, height))
#                     for image in files:
#                         video.write(cv2.imread(image))
#                     video.release()
                    
            print('Elapsed time:', time.time() - now, 'sec')
            print('Num. processed file:' + str(cnt) + '/' + str(len(files)))
            print('Total Elapsed time:', (time.time() - start_time)/3600, 'hrs')
            
            # save content as json
            json_name = json_path + fname.split('/')[-1][:-3] + 'json'
            
            if not exists(json_name):
                content2 = c['instances'][j]
                content2['gloss'] = c['gloss']
                json_object = json.dumps(content2, indent=4)
                # Writing to sample.json
                with open(json_name, "w") as outfile:
                    outfile.write(json_object)

#             if j == 4:
#                 break
#             break
        
#     break
    

Processing content 1/2000, instance 1/40
Elapsed time: 6.776068210601807 sec
Num. processed file:1/21095
Total Elapsed time: 0.001882840461201138 hrs
Processing content 1/2000, instance 2/40
Elapsed time: 6.629955053329468 sec
Num. processed file:2/21095
Total Elapsed time: 0.0037251475784513687 hrs
Processing content 1/2000, instance 3/40
Elapsed time: 6.0096495151519775 sec
Num. processed file:3/21095
Total Elapsed time: 0.00539515475432078 hrs
Processing content 1/2000, instance 4/40
Elapsed time: 12.115551710128784 sec
Num. processed file:4/21095
Total Elapsed time: 0.008761056065559387 hrs
Processing content 1/2000, instance 5/40
Elapsed time: 9.421323299407959 sec
Num. processed file:5/21095
Total Elapsed time: 0.011378979219330681 hrs
Processing content 1/2000, instance 6/40
Elapsed time: 9.56457805633545 sec
Num. processed file:6/21095
Total Elapsed time: 0.014036403761969673 hrs
Processing content 1/2000, instance 7/40
Elapsed time: 11.571477890014648 sec
Num. processed file:7

Elapsed time: 6.2034220695495605 sec
Num. processed file:56/21095
Total Elapsed time: 0.12362667712900374 hrs
Processing content 2/2000, instance 17/35
Elapsed time: 6.604982852935791 sec
Num. processed file:57/21095
Total Elapsed time: 0.12546228614118365 hrs
Processing content 2/2000, instance 18/35
Elapsed time: 4.219435930252075 sec
Num. processed file:58/21095
Total Elapsed time: 0.1266348284482956 hrs
Processing content 2/2000, instance 19/35
Elapsed time: 6.860560655593872 sec
Num. processed file:59/21095
Total Elapsed time: 0.1285412198967404 hrs
Processing content 2/2000, instance 20/35
Elapsed time: 5.7736687660217285 sec
Num. processed file:60/21095
Total Elapsed time: 0.13014599945810107 hrs
Processing content 2/2000, instance 21/35
Elapsed time: 4.062490940093994 sec
Num. processed file:61/21095
Total Elapsed time: 0.13127489282025231 hrs
Processing content 2/2000, instance 22/35
Elapsed time: 7.750144958496094 sec
Num. processed file:62/21095
Total Elapsed time: 0.1334282

Elapsed time: 2.713028907775879 sec
Num. processed file:111/21095
Total Elapsed time: 0.2222652597559823 hrs
Processing content 4/2000, instance 7/26
Elapsed time: 6.866310358047485 sec
Num. processed file:112/21095
Total Elapsed time: 0.2241734106010861 hrs
Processing content 4/2000, instance 8/26
Elapsed time: 8.012669563293457 sec
Num. processed file:113/21095
Total Elapsed time: 0.22640014588832855 hrs
Processing content 4/2000, instance 9/26
Elapsed time: 5.850732803344727 sec
Num. processed file:114/21095
Total Elapsed time: 0.2280257621076372 hrs
Processing content 4/2000, instance 10/26
Elapsed time: 7.850283622741699 sec
Num. processed file:115/21095
Total Elapsed time: 0.23020736826790703 hrs
Processing content 4/2000, instance 11/26
Elapsed time: 5.241337537765503 sec
Num. processed file:116/21095
Total Elapsed time: 0.23166407406330108 hrs
Processing content 4/2000, instance 12/26
Elapsed time: 3.8699190616607666 sec
Num. processed file:117/21095
Total Elapsed time: 0.23273

Elapsed time: 3.7076799869537354 sec
Num. processed file:166/21095
Total Elapsed time: 0.331494798262914 hrs
Processing content 6/2000, instance 10/26
Elapsed time: 6.920334577560425 sec
Num. processed file:167/21095
Total Elapsed time: 0.33341780033376484 hrs
Processing content 6/2000, instance 11/26
Elapsed time: 9.711901187896729 sec
Num. processed file:168/21095
Total Elapsed time: 0.33611622088485293 hrs
Processing content 6/2000, instance 12/26
Elapsed time: 9.690764904022217 sec
Num. processed file:169/21095
Total Elapsed time: 0.33880861302216847 hrs
Processing content 6/2000, instance 13/26
Elapsed time: 7.303552627563477 sec
Num. processed file:170/21095
Total Elapsed time: 0.34083813925584155 hrs
Processing content 6/2000, instance 14/26
Elapsed time: 4.207663297653198 sec
Num. processed file:171/21095
Total Elapsed time: 0.3420074297984441 hrs
Processing content 6/2000, instance 15/26
Elapsed time: 4.683371067047119 sec
Num. processed file:172/21095
Total Elapsed time: 0.34

Elapsed time: 4.548766851425171 sec
Num. processed file:221/21095
Total Elapsed time: 0.4407050958606932 hrs
Processing content 8/2000, instance 14/25
Elapsed time: 7.737685918807983 sec
Num. processed file:222/21095
Total Elapsed time: 0.44285508116086325 hrs
Processing content 8/2000, instance 15/25
Elapsed time: 5.4922404289245605 sec
Num. processed file:223/21095
Total Elapsed time: 0.44438159227371216 hrs
Processing content 8/2000, instance 16/25
Elapsed time: 11.128054141998291 sec
Num. processed file:224/21095
Total Elapsed time: 0.4474732419517305 hrs
Processing content 8/2000, instance 17/25
Elapsed time: 5.740789175033569 sec
Num. processed file:225/21095
Total Elapsed time: 0.44906876100434195 hrs
Processing content 8/2000, instance 18/25
Elapsed time: 5.595398187637329 sec
Num. processed file:226/21095
Total Elapsed time: 0.45062356942229803 hrs
Processing content 8/2000, instance 19/25
Elapsed time: 5.1002137660980225 sec
Num. processed file:227/21095
Total Elapsed time: 0

Elapsed time: 3.9495553970336914 sec
Num. processed file:276/21095
Total Elapsed time: 0.5265231042438083 hrs
Processing content 11/2000, instance 20/23
Elapsed time: 3.5065224170684814 sec
Num. processed file:277/21095
Total Elapsed time: 0.5274976401196586 hrs
Processing content 11/2000, instance 21/23
Elapsed time: 7.833075284957886 sec
Num. processed file:278/21095
Total Elapsed time: 0.5296743990977605 hrs
Processing content 11/2000, instance 22/23
Elapsed time: 8.016386985778809 sec
Num. processed file:279/21095
Total Elapsed time: 0.5319019286500083 hrs
Processing content 11/2000, instance 23/23
Elapsed time: 7.429438352584839 sec
Num. processed file:280/21095
Total Elapsed time: 0.5339666976531346 hrs


In [ ]:
with open(json_name) as ipf:
    temp_content = json.load(ipf)

In [ ]:
temp_content

In [21]:
from collections import Counter

[print(sum()) for c in content]
# Counter()

40
35
30
26
26
26
25
25
24
23
23
22
22
22
22
22
22
22
21
21
21
21
21
21
21
21
21
21
21
21
21
21
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
16
16
16
16
16
16
16
16
16
16
16
16
16
15
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
1

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [23]:
len(content[0]['instances'])

40